# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
from feature_selection import forwardFeatureSelection

sys.path.append("/Users/paolo/Documents/methods/LinCFA")
from LinCFA import LinCFA

sys.path.append("/Users/paolo/Documents/methods/NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("/Users/paolo/Documents/Droughts/Paolo/regression_NonLinCFA")
from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2


# Eps = 0.001

In [ ]:
### BOTH, with features from all basins
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Emiliani1','Emiliani2','Garda_Mincio']

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.001
    actual_path='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg'
    
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps, multiple=True,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
    #selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    #print('\nFull model and selected features with wrapper\n')
    #compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    #print('\nFull model and best 5 selected features with wrapper\n')
    #compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    #res = {
    #        "delta" : [], 
    #        "numSelected" : [], 
    #        "selectedFeatures" : [] 
    #    }
    
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    

####################Emiliani1####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1 -0.382765
1    2001-01-13  0.482679    0.58  2001     2  0.319215
2    2001-01-21  0.516259    0.59  2001     3  0.548542
3    2001-01-29  0.434421    0.50  2001     5 -0.010351
4    2001-02-06  0.494805    0.54  2001     6  0.402030
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.427085    0.43  2009    48 -0.060454
407  2009-12-05  0.547380    0.57  2009    49  0.761079
408  2009-12-13  0.531070    0.58  2009    50  0.649694
409  2009-12-21  0.295704    0.00  2009    52 -0.957702
410  2009-12-29  0.027861    0.00  2009    53 -2.786888

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 991



In [ ]:
### for the plot:
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Emiliani1']
path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

fig, ax = plt.subplots(2)
xmin=6.6
xmax=13
ymin=43.7
ymax=46.6
ax[0].set_xlim(xmin,xmax)
ax[1].set_xlim(xmin,xmax)
ax[0].set_ylim(ymin,ymax)
ax[1].set_ylim(ymin,ymax)

output_tot = []
### BOTH
for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
        
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
    output_tot.append(output)
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)
    
    plot_cells(output,selected_colnames)
        

####################Emiliani1####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.379890    0.50  2001     1 -0.382765
1    2001-01-13  0.482679    0.58  2001     2  0.319215
2    2001-01-21  0.516259    0.59  2001     3  0.548542
3    2001-01-29  0.434421    0.50  2001     5 -0.010351
4    2001-02-06  0.494805    0.54  2001     6  0.402030
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.427085    0.43  2009    48 -0.060454
407  2009-12-05  0.547380    0.57  2009    49  0.761079
408  2009-12-13  0.531070    0.58  2009    50  0.649694
409  2009-12-21  0.295704    0.00  2009    52 -0.957702
410  2009-12-29  0.027861    0.00  2009    53 -2.786888

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 172

Number of aggregated features: 11

Number of features: 172

Number of aggregated features: 10

Number of features: 172

Number of aggregated features: 12

N

# Altro

In [42]:
best_epsilon_wrapper = []
best_epsilon_wrapper_best5 = []
best_epsilon_CMI = []
best_epsilon_CMI_best5 = []

for basin in basins:
    current_max = max([ v for k, v in val_res_wrapper.items() if k.startswith(basin)])
    best_epsilon_wrapper.append([ [k,v] for k, v in val_res_wrapper.items() if v==current_max][0][0])
    
    current_max = max([ v for k, v in val_res_wrapper_best5.items() if k.startswith(basin)])
    best_epsilon_wrapper_best5.append([ [k,v] for k, v in val_res_wrapper_best5.items() if v==current_max][0][0])
    
    current_max = max([ v for k, v in val_res_CMI.items() if k.startswith(basin)])
    best_epsilon_CMI.append([ [k,v] for k, v in val_res_CMI.items() if v==current_max][0][0])
    
    current_max = max([ v for k, v in val_res_CMI_best5.items() if k.startswith(basin)])
    best_epsilon_CMI_best5.append([ [k,v] for k, v in val_res_CMI_best5.items() if v==current_max][0][0])
    

In [5]:
### BOTH
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in ['Emiliani2']:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')

    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
        
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2010-01-01', max_val='2015-01-01', max_test='2020-01-01')
        
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 228)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    

####################Emiliani2####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.214281    0.00  2001     1 -1.339879
1    2001-01-13  0.484737    0.52  2001     2  0.402993
2    2001-01-21  0.466071    0.47  2001     3  0.282703
3    2001-01-29  0.417470    0.44  2001     5 -0.030490
4    2001-02-06  0.492202    0.53  2001     6  0.451097
..          ...       ...     ...   ...   ...       ...
406  2009-11-27  0.436464    0.46  2009    48  0.091910
407  2009-12-05  0.466152    0.49  2009    49  0.283224
408  2009-12-13  0.553659    0.59  2009    50  0.847138
409  2009-12-21  0.507978    0.65  2009    52  0.552758
410  2009-12-29  0.083046    0.00  2009    53 -2.185583

[411 rows x 6 columns]
 target shapes: ((411, 6), (228, 6), (639, 6), (228, 6))
Number of features: 130

Number of aggregated features: 7

Number of features: 130

Number of aggregated features: 8

Number of features: 130

Number of aggregated features: 8

Numb

In [17]:
aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_wrapper_best5_train.csv', index=False)
aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_wrapper_best5_val.csv', index=False)
aggregate_test.loc[:,selected_colnames[0:5]].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_wrapper_best5_test.csv', index=False)


In [18]:
res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
    
selectedFeatures='selectedFeatures'
print(f'\n{res[selectedFeatures]}\n')
selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]

print('\nFull model and selected features with CMI\n')
compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)

print('\nFull model and best 5 selected features with CMI\n')
compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        

----- MI Scores -----
[(11, 0.15246918228231238), (62, 0.13685487884232336), (73, 0.13524591194316204), (4, 0.1349170730411436), (12, 0.13069123294496232), (2, 0.13065462984779644), (10, 0.1231135469682487), (0, 0.12179110241277581), (1, 0.11185209883590831), (8, 0.10986275099519403), (65, 0.09926803152018446), (3, 0.09837955367460148), (6, 0.09674503527089101), (66, 0.0966209825430217), (75, 0.09623197123570164), (72, 0.09324017948760668), (79, 0.09207223934751455), (67, 0.09104982381307869), (74, 0.09054955462491504), (22, 0.09003186611152561), (71, 0.08873753149051027), (13, 0.08840980007656025), (63, 0.08674288030600526), (5, 0.08633953056624281), (42, 0.08551713286237457), (7, 0.08332675656174225), (70, 0.08179196380125643), (41, 0.08158274429258124), (84, 0.08130184534173333), (38, 0.07919816591699666), (78, 0.07608865743150646), (81, 0.07593527481570693), (49, 0.07551138688433962), (51, 0.07515073162512392), (61, 0.07453154808910913), (45, 0.07370388129928994), (64, 0.0727260798

0.23062800854005472

In [19]:
aggregate_trainVal.loc[:410,:].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_CMI_train.csv', index=False)
aggregate_trainVal.loc[411:,:].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_CMI_val.csv', index=False)
aggregate_test.loc[:,:].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_CMI_test.csv', index=False)


In [20]:
aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_CMI_best5_train.csv', index=False)
aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_CMI_best5_val.csv', index=False)
aggregate_test.loc[:,selected_colnames[0:5]].to_csv('/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/Emiliani2_nonLinCFA_CMI_best5_test.csv', index=False)
